# Julia言語で入門するプログラミング

[Julia言語で入門するプログラミング](https://muuumin.net/julia-intro-prog/)

## ディスパッチによる処理分岐

前回のプログラムでは、プレイヤーかモンスターかによって行動選択の処理を条件分岐で分けていた

以下の部分である

```julia
function 行動!(攻撃者::キャラクター, 防御者::キャラクター; io::IO = stdout)
    if 攻撃者.isプレイヤー
        # プレイヤーキャラクターの場合は行動入力
        コマンドID = Base.prompt("[1]攻撃 [2]大振り")
        攻撃処理!(攻撃者, 防御者, 行動コマンド[コマンドID]; io)
    else
        攻撃処理!(攻撃者, 防御者, :攻撃; io)
    end
end
```

このような条件分岐による処理分岐は、条件の種類が増える度に条件分岐を追記していかなければならないため、あまりメンテナンス性が良くない

これを解決するための機能として、Julia には **多重ディスパッチ** という非常に強力な仕組みが備わっている

ディスパッチとは、関数に渡される引数の型によって、同名の関数から適切な関数に自動で振り分ける仕組みである

分かりやすい例でいうと、Julia には `*` という演算子があるが、これは以下のような関数として定義されている

```julia
# 整数同士の乗算: 乗算処理
(*)(a::Int, b::Int) = a * b

# 文字列同士の乗算: 結合処理
(*)(a::String, b::String) = string(a, b)
```

上記のように同名の関数が複数存在する場合、Julia は関数に渡された引数の型を調べて、自動的に適切な関数を実行する

```julia
# (*)(2::Int, 3::Int) -> a * b が実行される
2 * 3 # => 6

# (*)("Hello"::String, "World"::String) -> string(a, b) が実行される
"Hello" * "World" # => "HelloWorld"

# (*)("Hello"::String, 10::Int) の関数は定義されていない
"Hello" * 10 # => 例外発生
```

この機能を使って、プレイヤー｜モンスターの行動選択処理の分岐を記述してみる

ディスパッチ機能を使うには、異なる処理には異なる型を関連付ける必要があるため、プレイヤーとモンスターの構造体をそれぞれ分けて定義する

In [1]:
"再定義可能な構造体を定義するマクロ"
macro restruct(構造体宣言式)
    一時的な構造体の名前 = "var\"$(gensym())\""
    mutable = 構造体宣言式.args[1] ? "mutable" : ""
    構造体を束縛する変数名 = 構造体宣言式.args[2]
    構造体内部コード = join(構造体宣言式.args[3].args, ";")

    Meta.parse(
        "Base.@kwdef $mutable struct $一時的な構造体の名前 $構造体内部コード end; $構造体を束縛する変数名 = $一時的な構造体の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

"""
キャラクター構造体をプレイヤー構造体とモンスター構造体に分ける
"""
# プレイヤー構造体
@restruct mutable struct Player
    名前::String = "勇者"
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

# モンスター構造体
@restruct mutable struct Monster
    名前::String = "モンスター"
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

var"##292"

構造体を分けて定義したら、それぞれの構造体に紐づく行動選択処理をそれぞれ定義し、ディスパッチによる分岐をできるようにする

In [2]:
"コマンドシンボル対応表"
行動コマンド = Dict(
    "1" => :攻撃,
    "2" => :大振り
)

"""
    行動選択(::Player) -> 行動シンボル::Symbol

行動選択処理 for Player
* 定義されていないコマンドを入力した場合は例外発生
"""
行動選択(::Player) = begin
    # 標準入力による行動選択
    コマンドID = Base.prompt("[1]攻撃 [2]大振り")
    行動コマンド[コマンドID]
end

"""
    行動選択(::Monster) -> 行動シンボル::Symbol

行動選択処理 for Monster
* 常に :攻撃 が選択される
"""
行動選択(::Monster) = :攻撃

行動選択

In [3]:
# プレイヤーに対する行動選択
## => 標準入力で入力されたコマンドに応じて行動が選択される
勇者 = Player()
行動選択(勇者)

:大振り

In [4]:
# モンスターに対する行動選択
## => 常に :攻撃 が選択される
モンスター = Monster()
行動選択(モンスター)

:攻撃

上記の通り、条件分岐を用いずに処理を分けることが出来た

現状では、キャラクターの種類が2種類しかいないためあまりシンプルになった気はしないかもしれないが、今後キャラクターの種類がどんどん増えていった場合、単純に型定義と型に紐づく関数をどんどん量産していくだけで良いため、非常に柔軟性の高いコードになっている

## Holy Traits

さて、キャラクターをプレイヤーとモンスターに分けたことで一つ問題が発生している

それは、プレイヤーとモンスターに共通する処理が書けなくなっていることである

例えば、どちらが先に攻撃するかを決定する `先行後攻決定(勇者::キャラクター, モンスター::キャラクター)` 関数などは、プレイヤーとモンスターが同一の型であることが前提となっている

```julia
先行後攻決定(勇者::キャラクター, モンスター::キャラクター) = if rand() < 0.5
        [(勇者, モンスター), (モンスター, 勇者)]
    else
        [(モンスター, 勇者), (勇者, モンスター)]
    end
```

こういった関数の場合、それぞれの型について同じ関数を全部定義していくのはあまり現実的ではない

（現状では、プレイヤーとモンスターしかいないが、今後キャラクターが追加される度に、`先行後攻決定` 関数をコピペしなければならなくなる）

これを解決するための手法として、Tim Holy 氏が提案した [Traits パターン](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633) がある

Holy 氏が提案したため、Holy Traits パターンと呼ばれるこの手法は非常にシンプルで、共通処理に対する型をさらに別途定義するというものである

このような共通処理用の型を **Trait** と呼ぶ

今回の場合だと、`キャラクター` 構造体を Trait として 定義すれば良い

In [5]:
# 中身のないキャラクター構造体を Trait として定義
## 共通処理関数のディスパッチのためだけに型定義
@restruct struct キャラクター end

# 共通処理関数を キャラクター Trait に対して定義
"""
    先行後攻決定(勇者::キャラクター, モンスター::キャラクター)
        -> [(攻撃者::キャラクター, 防御者::キャラクター), (攻撃者::キャラクター, 防御者::キャラクター)]

先攻後攻をランダムに決定する
"""
先行後攻決定(勇者::キャラクター, モンスター::キャラクター) = if rand() < 0.5
    [(勇者, モンスター), (モンスター, 勇者)]
else
    [(モンスター, 勇者), (勇者, モンスター)]
end

先行後攻決定

Trait と Trait に対するディスパッチ関数を定義したら、共通処理関数に渡したい型を対象の Trait 型に変換する関数を定義すれば良い

今回の場合だと、`Player` 構造体と `Monster` 構造体それぞれを `キャラクター` 構造体に変換する関数を定義すれば良い

それぞれの型に対して変換関数を書いても良いが、`Player` と `Monster` を包括する `Union` 型を引数にするとシンプルに書ける

In [6]:
"Player, Monster => キャラクター Trait 変換関数"
toキャラクター(::Union{Player, Monster}) = キャラクター()

toキャラクター

In [7]:
# 共通処理関数に渡したい場合は、対象の Trait 型に変換して渡せば良い
先行後攻決定(toキャラクター(勇者), toキャラクター(モンスター))

2-element Vector{Tuple{var"##293", var"##293"}}:
 (var"##293"(), var"##293"())
 (var"##293"(), var"##293"())

### Trait 型から派生型への変換
現状の Trait 型（キャラクター）と 派生型（Player, Monster）の関係を再掲すると、以下のように並列関係になっている

```julia
# キャラクター Trait
struct キャラクター end

# Player 構造体
struct Player
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

# Monster 構造体
struct Monster
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

"""
# 型階層

- キャラクター
- Player
- Monster
"""
```

実はこの型関係には問題がある

`先行後攻決定` 関数で `キャラクター` Trait のタプル配列を取得した後、`行動選択` 関数に渡す際、`キャラクター` Trait から `Player` or `Monster` 型に変換する手段がないのである

```julia
勇者 = Player("勇者", 30, 10, 10)
モンスター = Monster("モンスター", 30, 10, 10)

[
    (攻撃者1::キャラクター, 防御者1::キャラクター),
    (攻撃者2::キャラクター, 防御者2::キャラクター)
] = 先行後攻決定(
    toキャラクター(勇者) # Player => キャラクター への型変換
    toキャラクター(モンスター) # Monster => キャラクター への型変換
)

行動選択(
    # 攻撃者1::キャラクター の元の型が Player なのか Monster なのか分からない
    # => 変換できない
    toPlayer(攻撃者1)? or toMonster(攻撃者1)?
)
```

### 抽象型と型階層
上記の問題を解決するために、キャラクター Trait を **抽象型** に格上げし、Player と Monster は、キャラクター抽象型の継承型として定義する

```julia
abstract type キャラクター end

mutable struct Player <: キャラクター
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

mutable struct Monster <: キャラクター
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

"""
# 型階層

キャラクター
 |_ Player
 |_ Monster
"""
```

抽象型とは `abstract type` で宣言される型で、他の型の親として設定することができる（継承することができる）

ただし、中にフィールドデータを持ったり、インスタンス化したりすることはできない

一方、`struct` 等の型は具象型と呼ばれ、フィールドデータを持ったりインスタンス化することはできるが、他の型の親として設定することはできない（継承することが出来ない）

さて、このような親子関係を作ることによって何が変わったかというと、引数として `::キャラクター` 型を渡す共通関数に対して、`Player` や `Monster` のインスタンスを渡せるようになるのである

これにより `Player` or `Monster` => `キャラクター` への型変換が不要になり、同時に `キャラクター` => `Player` or `Monster` への型復元も不要になるのである

In [8]:
"再定義可能な構造体を定義するマクロ（型継承対応版）"
macro restruct(構造体宣言式)
    一時的な構造体の名前 = "var\"$(gensym())\""
    mutable = 構造体宣言式.args[1] ? "mutable" : ""
    構造体を束縛する変数名, 継承式 = if typeof(構造体宣言式.args[2]) === Expr
        (構造体宣言式.args[2].args[1], "<: $(構造体宣言式.args[2].args[2])")
    else
        (構造体宣言式.args[2], "")
    end
    構造体内部コード = join(構造体宣言式.args[3].args, ";")

    Meta.parse(
        "Base.@kwdef $mutable struct $一時的な構造体の名前 $継承式 $構造体内部コード end; $構造体を束縛する変数名 = $一時的な構造体の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

"再定義可能な型を定義するマクロ"
macro retype(型宣言式)
    一時的な型の名前 = "var\"$(gensym())\""
    型を束縛する変数名 = 型宣言式.args[1]

    Meta.parse(
        "abstract type $一時的な型の名前 end; $型を束縛する変数名 = $一時的な型の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

# キャラクター Trait (抽象型)
@retype abstract type キャラクター end

# プレイヤー構造体 <: キャラクター
@restruct mutable struct Player <: キャラクター
    名前::String = "勇者"
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

# モンスター構造体 <: キャラクター
@restruct mutable struct Monster <: キャラクター
    名前::String = "モンスター"
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

"""
    先行後攻決定(勇者::キャラクター, モンスター::キャラクター)
        -> [(攻撃者::キャラクター, 防御者::キャラクター), (攻撃者::キャラクター, 防御者::キャラクター)]

先攻後攻をランダムに決定する
"""
先行後攻決定(勇者::キャラクター, モンスター::キャラクター) = if rand() < 0.5
    [(勇者, モンスター), (モンスター, 勇者)]
else
    [(モンスター, 勇者), (勇者, モンスター)]
end

"コマンドシンボル対応表"
行動コマンド = Dict(
    "1" => :攻撃,
    "2" => :大振り
)

"""
    行動選択(::Player) -> 行動シンボル::Symbol

行動選択処理 for Player
* 定義されていないコマンドを入力した場合は例外発生
"""
行動選択(::Player) = begin
    # 標準入力による行動選択
    コマンドID = Base.prompt("[1]攻撃 [2]大振り")
    行動コマンド[コマンドID]
end

"""
    行動選択(::Monster) -> 行動シンボル::Symbol

行動選択処理 for Monster
* 常に :攻撃 が選択される
"""
行動選択(::Monster) = :攻撃

行動選択

In [9]:
勇者 = Player()
モンスター = Monster()

# 先攻後攻をランダムに決定
## 勇者::Player <: キャラクター, モンスター::Monster <: キャラクター のため
## 先行後攻決定(::キャラクター, ::キャラクター) 関数にそのまま渡すことができる
攻撃順 = 先行後攻決定(勇者, モンスター)

2-element Vector{Tuple{var"##294", var"##294"}}:
 (var"##295"("勇者", 30, 10, 10), var"##296"("モンスター", 30, 10, 10))
 (var"##296"("モンスター", 30, 10, 10), var"##295"("勇者", 30, 10, 10))

In [10]:
# 行動順に行動選択
for (攻撃者, 防御者) in 攻撃順
    行動 = 行動選択(攻撃者)
    println("$(攻撃者.名前) の $行動 ！")
end

勇者 の 大振り ！
モンスター の 攻撃 ！


### パラメトリック型
抽象型 (abstract type) は、型階層構造を構築するために強力で便利な仕組みである

一方、内部にフィールドデータ（メンバ変数）を持つことができないという制約がある

上記の場合では、`キャラクター` 抽象型にフィールドデータは持てないため、`Player`, `Monster` 構造体それぞれにフィールドデータを持たせていた

しかし、`Player` も `Monster` も持たせたいフィールドデータは同一であるため、本当は `キャラクター` 構造体に共通するフィールドデータを持たせたいのである

```julia
# キャラクター構造体に共通のフィールドデータを持たせたい
mutable struct キャラクター
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

# この場合 キャラクター は抽象型でないので以下のように書くことは出来ないが、
# 本当はこのような型の関係性を構築したい
struct Player <: キャラクター end
struct Monster <: キャラクター end

行動選択(::Player) = プレイヤー用の行動選択関数()
行動選択(::Monster) = モンスター用の行動選択関数()
```

これを Holy Traits パターンで実現するためには、`キャラクター` 型のオブジェクトを `Player` or `Monster` 型に変換するための戦略が必要である

すなわち、「`キャラクター` 構造体に `Player`, `Monster` 型のいずれの型に属するかの情報を内包できれば良い」のである

これを実現するための仕組みとして **パラメトリック型** が存在する

パラメトリック型は基本的に以下のような形で宣言する

```julia
# 任意の型 T に関する `キャラクター` パラメトリック型
mutable struct キャラクター{T}
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end
```

この仕組みを使うことで、`キャラクター` 型を `Player` or `Monster` 型に変換できるようになる

In [11]:
"再定義可能な構造体を定義するマクロ（パラメトリック型対応版）"
macro restruct(構造体宣言式)
    一時的な構造体の名前 = "var\"$(gensym())\""
    mutable = 構造体宣言式.args[1] ? "mutable" : ""
    構造体を束縛する変数名, 型付随情報 = if typeof(構造体宣言式.args[2]) === Expr
        if 構造体宣言式.args[2].head === :curly
            (構造体宣言式.args[2].args[1], "{$(構造体宣言式.args[2].args[2])}")
        else
            (構造体宣言式.args[2].args[1], "<: $(構造体宣言式.args[2].args[2])")
        end
    else
        (構造体宣言式.args[2], "")
    end
    構造体内部コード = join(構造体宣言式.args[3].args, ";")

    Meta.parse(
        "Base.@kwdef $mutable struct $(一時的な構造体の名前)$(型付随情報) $構造体内部コード end; $構造体を束縛する変数名 = $一時的な構造体の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

# 任意型 T に関する `キャラクター` 構造体
@restruct mutable struct キャラクター{T}
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

# 行動選択の振る舞いを変えるための Player Trait
@restruct struct Player end

# 行動選択の振る舞いを変えるための Monster Trait
@restruct struct Monster end

"キャラクター構造体を Player or Monster 型に変換する関数"
# where 型制約により キャラクター{Player} or キャラクター{Monster} のみ変換可能とする
to_player_or_monster(::キャラクター{T}) where T <: Union{Player, Monster} = T()

to_player_or_monster (generic function with 1 method)

In [12]:
to_player_or_monster(キャラクター{Player}("勇者", 30, 10, 10))

var"##298"()

In [13]:
to_player_or_monster(キャラクター{Monster}("モンスター", 30, 10, 10))

var"##299"()

In [14]:
# Player or Monster 以外の型に関するキャラクター型は変換不可
to_player_or_monster(キャラクター{Int}("Integer", 30, 10, 10))

LoadError: MethodError: no method matching to_player_or_monster(::var"##297"{Int64})
[0mClosest candidates are:
[0m  to_player_or_monster([91m::var"##297"{T}[39m) where T<:Union{var"##298", var"##299"} at In[11]:37

パラメトリック型を使った Holy Traits パターンにより、フィールドデータの共通化と、型による処理分岐を両方実現することができる

一通りの定義を以下にまとめてみる

In [20]:
# 任意型 T に関する `キャラクター` 構造体
@restruct mutable struct キャラクター{T}
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

# 行動選択の振る舞いを変えるための Player Trait
@restruct struct Player end

# 行動選択の振る舞いを変えるための Monster Trait
@restruct struct Monster end

"キャラクター構造体を Player or Monster 型に変換する関数"
# where 型制約により キャラクター{Player} or キャラクター{Monster} のみ変換可能とする
to_player_or_monster(::キャラクター{T}) where T <: Union{Player, Monster} = T()

"""
    先行後攻決定(勇者::キャラクター{T}, モンスター::キャラクター{T})
        -> [(攻撃者::キャラクター{T}, 防御者::キャラクター{T}), (攻撃者::キャラクター{T}, 防御者::キャラクター{T})]

先攻後攻をランダムに決定する
* キャラクター構造体は任意の型 P, M に関する構造体
* 勇者とモンスターは異なる型 (Player, Monster) に関するキャラクターになるはずなので
  パラメトリック型のパラメーター変数も異なる変数名にしておく
"""
先行後攻決定(勇者::キャラクター{P}, モンスター::キャラクター{M}) where P where M =
    if rand() < 0.5
        [(勇者, モンスター), (モンスター, 勇者)]
    else
        [(モンスター, 勇者), (勇者, モンスター)]
    end

"コマンドシンボル対応表"
行動コマンド = Dict(
    "1" => :攻撃,
    "2" => :大振り
)

"""
    行動選択(::Player) -> 行動シンボル::Symbol

行動選択処理 for Player
* 定義されていないコマンドを入力した場合は例外発生
"""
行動選択(::Player) = begin
    # 標準入力による行動選択
    コマンドID = Base.prompt("[1]攻撃 [2]大振り")
    行動コマンド[コマンドID]
end

"""
    行動選択(::Monster) -> 行動シンボル::Symbol

行動選択処理 for Monster
* 常に :攻撃 が選択される
"""
行動選択(::Monster) = :攻撃

行動選択

In [21]:
勇者 = キャラクター{Player}("勇者", 30, 10, 10)
モンスター = キャラクター{Monster}("モンスター", 30, 10, 10)

# 先攻後攻をランダムに決定
## 勇者::キャラクター{Player}, モンスター::キャラクター{Monster} を引数に渡された時点で
## 先行後攻決定(::キャラクター{Player}, ::キャラクター{Monster}) 関数が生成されて実行される
攻撃順 = 先行後攻決定(勇者, モンスター)

2-element Vector{Tuple{var"##312", var"##312"}}:
 (var"##312"{var"##314"}("モンスター", 30, 10, 10), var"##312"{var"##313"}("勇者", 30, 10, 10))
 (var"##312"{var"##313"}("勇者", 30, 10, 10), var"##312"{var"##314"}("モンスター", 30, 10, 10))

In [22]:
# 行動順に行動選択
for (攻撃者, 防御者) in 攻撃順
    # 行動選択関数の実行を to_player_or_monster 関数を介して行う
    行動 = 行動選択(to_player_or_monster(攻撃者))
    println("$(攻撃者.名前) の $行動 ！")
end

モンスター の 攻撃 ！
勇者 の 大振り ！


## まとめ

今回の型システムに関する話をまとめてコードに起こし、テストを実行する

テストコードは基本的に [前回のもの](./04_01/RPG/test/runtests.jl) をそのまま流用する

ただし、プレイヤーの行動選択処理である `行動入力処理()` は `行動選択処理(::プレイヤー)` に、モンスターの行動選択処理である `行動選択処理()` は `行動選択処理(::モンスター)` に変更する

これにより、行動選択処理を共通化することができるのである

In [24]:
using Pkg

if !isdir("./04_02")
    Pkg.generate("./04_02/RPG")
end

Pkg.activate("./04_02/RPG")
Pkg.add("Test")

  Generating  project RPG:
    ./04_02/RPG\Project.toml
    ./04_02/RPG\src/RPG.jl
  Activating project at `d:\github\julia_ml-tuto\01_tutorial\RPG\04_02\RPG`
    Updating registry at `C:\Users\user\.julia\registries\General.toml`
   Resolving package versions...
    Updating `D:\github\julia_ml-tuto\01_tutorial\RPG\04_02\RPG\Project.toml`
  [8dfed614] + Test
    Updating `D:\github\julia_ml-tuto\01_tutorial\RPG\04_02\RPG\Manifest.toml`
  [2a0f44e3] + Base64
  [b77e0a4c] + InteractiveUtils
  [56ddb016] + Logging
  [d6f4376e] + Markdown
  [9a3f8284] + Random
  [ea8e919c] + SHA
  [9e88b42a] + Serialization
  [8dfed614] + Test
Precompiling project...
  ✓ RPG
  1 dependency successfully precompiled in 1 seconds


In [26]:
# 自動テスト実行
Pkg.test()

Test Summary: | Pass  Total
戦闘開始処理  |    1      1
Test Summary: | Pass  Total
戦闘終了処理  |    3      3
Test Summary: | Pass  Total
先行後攻決定処理 |    2      2
Test Summary:   | Pass  Total
行動選択処理(::プレイヤー) |    3      3
Test Summary:   | Pass  Total
行動選択処理(::モンスター) |    1      1
Test Summary: | Pass  Total
行動メッセージ処理 |    1      1
Test Summary: | Pass  Total
ダメージメッセージ処理 |    2      2
Test Summary: | Pass  Total
攻撃ダメージ計算処理 |    8      8
Test Summary: | Pass  Total
ダメージ処理! |    5      5
Test Summary: | Pass  Total
大振り攻撃処理! |    4      4


     Testing RPG
      Status `C:\Users\user\AppData\Local\Temp\jl_m5vbSy\Project.toml`
  [67bf64dc] RPG v0.1.0 `D:\github\julia_ml-tuto\01_tutorial\RPG\04_02\RPG`
  [8dfed614] Test `@stdlib/Test`
      Status `C:\Users\user\AppData\Local\Temp\jl_m5vbSy\Manifest.toml`
  [67bf64dc] RPG v0.1.0 `D:\github\julia_ml-tuto\01_tutorial\RPG\04_02\RPG`
  [2a0f44e3] Base64 `@stdlib/Base64`
  [b77e0a4c] InteractiveUtils `@stdlib/InteractiveUtils`
  [56ddb016] Logging `@stdlib/Logging`
  [d6f4376e] Markdown `@stdlib/Markdown`
  [9a3f8284] Random `@stdlib/Random`
  [ea8e919c] SHA `@stdlib/SHA`
  [9e88b42a] Serialization `@stdlib/Serialization`
  [8dfed614] Test `@stdlib/Test`
     Testing Running tests...
     Testing RPG tests passed 
